In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver

# if you don't have GOOGLE_API_KEY in .env, set it manually here:
# os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY_HERE"

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3,
)

memory = SqliteSaver.from_conn_string(":memory:")

print("✅ ChatGoogleGenerativeAI + SqliteSaver imported and initialized.")

✅ ChatGoogleGenerativeAI + SqliteSaver imported and initialized.


In [15]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
import google.generativeai as genai
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.sqlite import SqliteSaver

In [4]:
tool = TavilySearchResults(max_results=2)

C:\Users\Chaudhary Computers\AppData\Local\Temp\ipykernel_10264\4289725543.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)


In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [6]:
import sys
print(sys.executable)


e:\conda_envs\langgraphenv\python.exe


In [7]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")
print("It worked!")

It worked!


In [18]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        # register the LLM node and the action node
        graph.add_node("llm", self.call_genai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_genai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def exists_action(self, state: AgentState):
        # safely check for tool_calls on the last message
        result = state["messages"][-1]
        tool_calls = getattr(result, "tool_calls", None)
        return bool(tool_calls and len(tool_calls) > 0)

    def take_action(self, state: AgentState):
        last_msg = state["messages"][-1]
        tool_calls = getattr(last_msg, "tool_calls", []) or []
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t["name"]].invoke(t.get("args", {}))
            results.append(ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result)))
        print("Back to the model!")
        return {"messages": results}

In [19]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

prompt = """You are a smart research assistant..."""
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)


In [20]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [21]:
thread = {"configurable": {"thread_id": "1"}}

In [41]:
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")
print("memory type:", type(memory))


memory type: <class 'contextlib._GeneratorContextManager'>


In [42]:
abot = Agent(
    model=model,
    tools=[tool],
    system=prompt,
    checkpointer=memory,
)


In [45]:
print("memory type =", type(memory))

checkpointer_attr = getattr(abot.graph, "checkpointer", None)
print("graph.checkpointer type =", type(checkpointer_attr))

print("abot.graph.invoke works? trying now...")
try:
    out = abot.graph.invoke({"messages": [HumanMessage(content="test")]}, {"configurable": {"thread_id": "X"}})
    print("invoke returned:", out)
except Exception as e:
    print("invoke error:", e)


memory type = <class 'contextlib._GeneratorContextManager'>
graph.checkpointer type = <class 'contextlib._GeneratorContextManager'>
abot.graph.invoke works? trying now...
invoke error: '_GeneratorContextManager' object has no attribute 'get_next_version'


In [47]:
abot_no_mem = Agent(
    model=model,
    tools=[tool],
    system=prompt,
    checkpointer=None,   # <-- yahan change dekho
)

print("BOT SAYS:", abot_no_mem.invoke("hello without memory", thread_id="1"))


AttributeError: 'Agent' object has no attribute 'invoke'

In [46]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

AttributeError: '_GeneratorContextManager' object has no attribute 'get_next_version'

In [ ]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

In [ ]:
#Streaming tokens

from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [ ]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")